* clean monetary values
* select Dublin only
* RegExp: Extract Postcode
* RegExp: Area Mapping
  * use place names file to extract core location
  * use Area mapping file to get postalcode
  
<br>
Results:

* total nbr rows: 5377
* regexp part 1
  * nbr rows w/extrated regexp part 1: 2183
  * nbr rows remaining w/unknown zipcode_new: 3194
* regexp part 2
  * nbr rows w/unknown zipcode_new after regexp part 2: 2193
    * nbr rows completed through area mapping: 1001
    * Entire home/apt nbr rows w/unknown zipcode_new: 1087

In [1]:
import pandas as pd
import numpy as np
import csv
import re
from collections import defaultdict
import requests

pd.set_option('display.expand_frame_repr', False)

### Read Data

In [2]:
# Listings

# %cd ../../data/airbnbdata
# filename = '201702_listings.csv.gz'
# filename = '201702_listings.csv'
filename = 'http://data.insideairbnb.com/ireland/leinster/dublin/2017-02-18/data/listings.csv.gz'
df_read = pd.read_csv(filename,
                       encoding='utf-8',
                       compression='gzip',
                       sep=',',
                       index_col = 0,
                       header=0
                       )
df_read['street'] = df_read['street'].apply(lambda x: x.lower())
df_read.head(1)

,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,
10778114,https://www.airbnb.com/rooms/10778114,20170218121908,2017-02-18,1 Bedroom Apartment For Short Term,Contact for more info,- Kitchen - Wifi - PS4,Contact for more info - Kitchen - Wifi - PS4 1...,none,NaN,NaN,...,NaN,f,NaN,NaN,f,flexible,f,f,1,NaN


### Load resource files

In [3]:
%pwd

'/usr/local/bin/notebooks/notebooks/airbnb'

In [4]:
# Places Names

%cd ../../data/airbnbdata/geo/regexp

f=open("pattern.txt", "r")
patterns =f.read().replace('\n','').lower()

# patterns
# type(patterns)

/usr/local/bin/notebooks/data/airbnbdata/geo/regexp


In [5]:
# Area Mapping

AreaDict = defaultdict(list)
with open("area_mapping.txt") as f:
    reader = csv.reader(f, delimiter=':')
    for line in reader:
         AreaDict[line[0].strip().lower()] = line[1].strip().lower()

# print(AreaDict.items())
# print(AreaDict['ashtown'])

### Pre-processing

In [6]:
df = df_read.copy()
print(df.shape)

(6729, 94)


#### Clean monetary values

In [7]:
df['price'] = df['price'].replace('[$,]','',regex=True).astype(float)
df['cleaning_fee'] = df['cleaning_fee'].replace('[$,]','',regex=True).astype(float)

#### Dublin City only

In [8]:
print(df.neighbourhood_cleansed.unique())
df.neighbourhood_cleansed.describe()
df[['street', 'zipcode', 'neighbourhood_cleansed']].isnull().sum(axis = 0)

['Dublin City' 'South Dublin' 'Fingal' 'Dn Laoghaire-Rathdown']


street                       0
zipcode                   4212
neighbourhood_cleansed       0
dtype: int64

In [9]:
df = df[df.neighbourhood_cleansed == 'Dublin City']
print(df.shape)
print(df.neighbourhood_cleansed.unique())
df.neighbourhood_cleansed.describe()
df[['street', 'zipcode']].isnull().sum(axis = 0)

(5377, 94)
['Dublin City']


street        0
zipcode    3305
dtype: int64

#### RegExp: Extract Postcode

In [10]:
"""
for reference, eircode regexp: re1 = re.search(r'd\d{1,2}\s{0,1}[a-z0-9]{4}', x) # eircode format Dxx xxxx

in this particular case however we can achieve the goal by a more general approach

below is the breakdown of \bd\d{2}|d\s\d{2}|d\d{1}w?|d\s\d{1,2}w?|d\d{1}\b

d\d{2} >> d01, d10
d\s\d{2} >> d 10, d 01
d\d{1}w? >> d6w
d\s\d{1}w? >> d 6w
d\d{1}\b >>  d1, d3

d(ublin)* >> same as above for dublin instead of d

"""
def replace(x):
    re1 = re.search(r'\bd\d{2}|d\s\d{2}|d\d{1}w?|d\s\d{1,2}w?|d\d{1}\b', x)
    re2 = re.search(r'\bd\dublin{2}|dublin\s\d{2}|dublin\d{1}w?|dublin\s\d{1,2}w?|dublin\d{1}\b', x)
#     x = x.strip()
    
    if re1:
        return 'dublin ' + re1[0][1:] 
    elif re2:
        if isinstance(re2[0][6], int):
            return 'dublin ' + re2[0][6:]
        else:
            return 'dublin ' + re2[0][7:]
    else:
        return ''

df['zipcode_new'] = (df['street'].apply(lambda x: replace(x)
                                        .replace('0', '')
                                        .replace('  ', ' ')
                                        .strip()))# if not pd.isnull(x) else np.nan))

df['zipcode_new'] = df['zipcode_new'].apply(lambda x: np.nan if x=='' else x)
df['zipcode_new'].unique()
# df = df.sample(frac=1)
# df[['street', 'zipcode_new']].sample(5)

array(['dublin 4', nan, 'dublin 2', 'dublin 6', 'dublin 8', 'dublin 1',
       'dublin 11', 'dublin 6w', 'dublin 3', 'dublin 7', 'dublin 5',
       'dublin', 'dublin 9', 'dublin 13', 'dublin 12', 'dublin 24',
       'dublin 15', 'dublin 17', 'dublin 35'], dtype=object)

In [11]:
print('total nbr rows: {}'.format(len(df)))
print('nbr rows w/extrated regexp part 1: {}'.format(len(df[~pd.isnull(df.zipcode_new)])))

regexp1 = len(df[pd.isnull(df.zipcode_new)])
print('nbr rows remaining w/unknown zipcode_new: {}'.format(regexp1))

total nbr rows: 5377
nbr rows w/extrated regexp part 1: 2183
nbr rows remaining w/unknown zipcode_new: 3194


#### RegExp Resource Files

- Step 1: use place names file to extract core location
- Step 2: use Area mapping file to get postalcode


In [12]:
def mapArea(x):
    res = AreaDict[x]
    if res:
        return res
    else:
        return np.nan
    
df = df.copy()

# create new column with extracted pattern (area name) if found
df['street_cleansed'] = df['street'].str.extract(r'('+patterns+')')

# get row id's of rows with unknown zipcode
mask = pd.isnull(df.zipcode_new) # boolean value will be set to true if isnull() 

# map extracted area name to get corresponding zipcode from resource file
df.loc[mask, 'zipcode_new'] = df[mask].apply(lambda row: mapArea(row.street_cleansed), axis=1)

df[['street', 'street_cleansed', 'zipcode_new']].head()

,street,street_cleansed,zipcode_new
id,,,
10778114,"south lotts road, dublin, dublin dublin 4, ire...",NaN,dublin 4
14348712,"dublin, leinster 1, ireland",NaN,NaN
15749806,"pinehurst, hainault road, foxrock, county dubl...",foxrock,dublin 18
9602076,"erne house, dublin, dublin 2, ireland",NaN,dublin 2
15952233,"gardiner street lower, dublin, county dublin, ...",NaN,NaN


In [18]:
regexp2 = len(df[pd.isnull(df.zipcode_new)])
print('nbr rows w/unknown zipcode_new after regexp part 2: {}'.format(regexp2))
print('nbr rows completed through area mapping: {}'.format(regexp1-regexp2))

# Quantify missing Postal code data for room_type=='Entire home/apt'
df_na = df[pd.isnull(df.zipcode_new)]
df_na = (df_na[(df_na.room_type=='Entire home/apt')])
print('Entire home/apt nbr rows w/unknown zipcode_new: {}'.format(len(df_na)))

nbr rows w/unknown zipcode_new after regexp part 2: 2193
nbr rows completed through area mapping: 1001
Entire home/apt nbr rows w/unknown zipcode_new: 1087


In [14]:
# see breakdown
# grouped = df_na.groupby('property_type').agg({'host_id': ['count']})
# grouped.columns = grouped.columns.map('_'.join)
# grouped.sort_values(by='host_id_count', ascending=False)

#### Write to file

In [15]:
# %cd ../../
# df.to_csv('201702_listings_preproc.csv', index=False)

#### Links

In [16]:
# https://rstudio-pubs-static.s3.amazonaws.com/407929_afc5ef0f2ad648389447a6ca3f4a7cd4.html